In [1]:
from IPython.display import clear_output
import pandas as pd
from pandas import Series
import numpy as np
import re
import csv
from gsheets import Sheets
from datetime import datetime

In [2]:
url = 'https://docs.google.com/spreadsheets/d/1LRbios7yQRo3aqCh0Es2Wiae_dicg_OtL-_yqP-Tb8I/edit#gid=1718343431'

## Create resusable lists for reference in data display or collection

In [3]:
# lists used in the loading of the AAT Metadata
terms_header_list = ["AACR2_FLAG", "DISPLAY_DATE", "DISPLAY_NAME", "DISPLAY_ORDER", "END_DATE", "HISTORIC_FLAG", "OTHER_FLAGS", "PREFERRED", "START_DATE", "SUBJECT_ID", "TERM", "TERM_ID", "VERNACULAR"]
terms_dtype = {'TERM_ID': str, 'SUBJECT_ID' : str}
lang_rels_header_list = ["LANGUAGE_CODE", "PREFERRED", "SUBJECT_ID", "TERM_ID", "QUALIFIER", "TERM_TYPE", "PART_OF_SPEECH", "LANG_STAT"]
lang_rel_dtype = {'TERM_ID': str, 'SUBJECT_ID' : str, 'LANGUAGE_CODE': str}
narrow_list = ['identifier', 'media', 'subject', 'title', 'subjects', 'SUBJECT_ID_x', 'TERM_ID', 'join_concept', 'BoxId', 'color']

## Functions

In [ ]:
%run functions.ipynb

## Load the spreadsheet information

In [ ]:
# if HttpAccessTokenRefreshError, close everything and start over
sheets = Sheets.from_files('~/client_secrets.json', '~/storage.json')
s = sheets.get(url)

In [ ]:
# get the list of boxes containing metadata

box_list = []
box_list_sheets = s.sheets.titles()

for b in box_list_sheets:
    if 'Box' in b:
        box_list.append(int(b[4:]))

## Load the Metadata from Getty AAT

Individual AAT Terms can be seen here:
https://www.getty.edu/research/tools/vocabularies/aat/

We use the .Out files available below for consuming the data - these are batch released so terms in the .Out files may note be availalbe on the AAT search (link above) or vice-versa:
http://aatdownloads.getty.edu/

Note that when we load files, we join the LANGUAGE_RELS.out and TERM.out files and combine the term and and the qualifier to create the "subjects" and "join_concepts"

The join_concept ensures the case for the term is all lower case

We also create a "full_concept" but doesn't handle the NULL case well and shouldn't be used for analysis

In [ ]:
concepts = load_metadata()

In [ ]:
# review the metadata for concepts containing a comma and return those as a list
concepts_drop = concepts[concepts['use_concept'].notna()]
concepts_search = concepts_drop[concepts_drop['use_concept'].str.contains(',')]
concepts_comma = concepts_search['use_concept'].to_list()

In [ ]:
concepts[concepts['join_concept'].notna()]

### Consider comma cases

given that we'll parse the created Arquin metadata by comma, we need to handle cases where the subject term, aka concept, contains an internal comma (e.g. "knives, gauge"). These will need to be parsed carfully for two reasons:

1.) The comma could falsely parse the concept

2.) The individual components of the concept could be valid AAT terms, too... eg. "knives, gauge" Both "knives" and "gauge" are valid AAT terms while so is "knives, gauge". Knowing what the metata analyst intended in this case is highly difficult to determine. 

In [ ]:
# display a sample of concepts_comma
concepts_comma[0:10]

In [ ]:
# identify the risk of concepts that have multiple commas in them

multi_commas_list = []
for concept in concepts_comma:
    x = concept.count(',')
    if x > 1:
        multi_commas_list.append(concept)

print(f'There are {len(multi_commas_list)} concepts that have more than 1 comma')

In [ ]:
# show a sample of the concepts with more than 1 comma
multi_commas_list[34:44]

## Begin parsing the subjects in the boxes

This process will parse the subjects and then evaluate them.

1. parse the subjects by comma (really a comma+space, ', ')
2. for all the subjects associated with a given media id, validate that they each match an AAT join_concept
3. If yes, move on to the next media id
4. If a parsed join_concept fails to match an AAT Term, identify the order of that term in the series of subjects (e.g. if there are 5 parsed terms in the set of 'subject', and 1, 2, and 5 match an AAT term). Then combine the nearest term with a subject (3 and 4 from the example) and see if that term matches AAT.
5. If yes (in 4), remove subjects 3 and 4 and replace with the combined 3 & 4. 
6. If no (in 4), log the unmatched terms for further analysis

In [ ]:
# box_list = [7]

In [ ]:
# load all the boxes and split them
for i, box_number in enumerate(box_list):
    df = return_box(box_number)
    clear_output(wait=True)
    print(f"working on Box {box_number}")
    
    try:
        df = split_df(df)
    except:
        print(f'failed to split df for box number {box_number}')

    if i+1 == 1:
        df_combined = df
    else:
        df_combined = combine_boxes(df_combined, df)
 
clear_output(wait=True)    

df = df_combined
df = df.drop(df[df['subjects'] == ''].index)
print(f'complete, imported {i+1} boxes')

In [ ]:
# join the df with the concepts
df = pd.merge(df, concepts, how="left", on='join_concept')

In [ ]:
# get the boxid
df['boxid_temp'] = df['media'].str.extract(r"(_0\d+_)")
df = df[df['media'].notna()]
df['boxid_temp'] = df['boxid_temp'].str.replace("_","")
df['boxid_temp'] = pd.to_numeric(df['boxid_temp'], errors='coerce')
df = df.dropna(subset=['boxid_temp'])
df['boxid_temp'] = df['boxid_temp'].astype('int')
df['boxid_temp'] = df['boxid_temp'].astype('str')
df['BoxId'] = 'Box '+df['boxid_temp']
df.drop('boxid_temp', inplace=True, axis=1)

## Add in Tab Color

The colors of each tab are identified in 'get_spreadsheets_color.ipynb'. That script creates a CSV file and that is read in and added to the df_combined object

Read in the color match file created here:
http://localhost:8888/notebooks/get_spreadsheets.ipynb

use that to create the list of titles based on color of the tab

In [ ]:
color_match_df = pd.read_csv('csv_files/color_match', sep='\t')

In [ ]:
df = pd.merge(df, color_match_df, on='BoxId')

In [ ]:
# df = df[df['media'] == 'B-1_Arquin_007_0257.jpg']
# df[narrow_list]

In [ ]:
# get all the unique media IDs where media is not NA 
df_medias = df.media.dropna().unique()

In [ ]:
df[df['identifier'] == 'A-1 259']

Run the full flow through in the following cell. Be sure to DELETE the unmatched_subjects_v2.csv file before staring

In [ ]:
df_medias_len = len(df_medias)

for i, search_media in enumerate(df_medias):
    clear_output(wait=True) 
    print(f'{i} of {df_medias_len}: search_media = {search_media}')
#     x = input('step 1: press any key to continue... ')
    
    try_match_neg = None
    try_match_pos = None
    cancel_repeat = False
    concepts_found_pos = []
    concepts_found_neg = []
    
    df_subjects = df[df['media'] == search_media].join_concept.to_list()
    df_media_no_match = df[df['media'] == search_media].loc[df['SUBJECT_ID_x'].isna()]
#     print(f'{i}: search_media = {search_media}, df_media_no_match = {df_media_no_match}')
#     x = input('press any key to continue... ')
    
    if len(df_media_no_match) > 0:
#         print(f'{i}: search_media = {search_media}, passed len(df_media_no_match) > 0 ')
#         x = input('step 2: press any key to continue... ')
        
        
        no_match = df_media_no_match.join_concept.to_list()
#         print(f'{i}: search_media: {search_media}, no_match = {no_match}')
        
        for i, unmatched in enumerate(no_match):
#             print(f'{i}: unmatched = {unmatched}, remove_con = {remove_con}')
#             x = input('step 3: press any key to continue... ')
#             if cancel_repeat == True:
#                 break
            
            if i > 0:
                if unmatched == remove_this_con:
#                     print(f'already removed {remove_con} from  {search_media}')
#                     x = input(f'step 4: it worked! Continue? (press x)')
#                     cancel_repeat = True
                    break
            
            pos = False
            neg = False
            new_concept = None
            pop_index = None
            
            no_match_index = df_subjects.index(unmatched)
#             print(f'step 5: {i+1}: unmatched concept is: "{unmatched}" and its index is "{no_match_index}" out of {len(df_subjects)-1}')
            
            if no_match_index > 0:
                try_match_neg = df_subjects[no_match_index-1]+', '+df_subjects[no_match_index]
                concepts_found_pos = concepts.loc[concepts['join_concept'] == try_match_neg]
                if len(concepts_found_pos) > 0:
                    pos = True
#                     print(f'pos is {pos}')
            if no_match_index < len(df_subjects)-1:
                try_match_pos = df_subjects[no_match_index]+', '+df_subjects[no_match_index+1]
                concepts_found_neg = concepts.loc[concepts['join_concept'] == try_match_pos]
                if len(concepts_found_neg) > 0:
                    neg = True
#                     print(f'neg is {neg}')
#             print(f'try_match_neg = "{try_match_neg}"\ntry_match_pos = "{try_match_pos}"')
            
            try:
                if len(concepts_found_pos) == 1:
                    new_concept = concepts_found_pos.join_concept.to_list()[0]
                    
                        
            except (NameError):
                pass
            try:
                if len(concepts_found_neg) == 1:
                    new_concept = concepts_found_neg.join_concept.to_list()[0]
            except (NameError):
                pass
            try:
                if (len(concepts_found_pos) > 0) & (len(concepts_found_neg) > 0):
                    print('too many matches!')
                    pass
            except (NameError):
                pass
            try:
                if (len(concepts_found_pos) == 0) & (len(concepts_found_neg) == 0):
                    unmatched_df = df[(df['media'] == search_media) 
                                      & (df['join_concept'] == unmatched)][narrow_list]
                    add_to_not_found(unmatched_df)
            except (NameError):
                pass
            
            remove_con = df_subjects[no_match_index]
            remove_this_con_index = 0
            
            if pos:
                remove_this_con_index = no_match_index-1
            if neg:
                remove_this_con_index = no_match_index+1

            if pos | neg:
                remove_this_con = df_subjects[remove_this_con_index]

                df = df.drop(
                    df[((df['media'] == search_media) & 
                                      (df['join_concept'] == remove_this_con)
                                     )].index
                )
            
    #     determine the location in the index of the concept to be removed
                pop_index = df.loc[(df['media'] == search_media) & (df['join_concept'] == remove_con)].index
    #     remove the concept found at that index
                df.at[pop_index[0], 'join_concept'] = new_concept
                df.at[pop_index[0], 'subjects'] = new_concept
                df.at[pop_index[0], 'SUBJECT_ID_x'] = concepts.at[concepts.loc[concepts['join_concept'] == new_concept].index[0], 'SUBJECT_ID_x']
                df.at[pop_index[0], 'TERM_ID'] = concepts.at[concepts.loc[concepts['join_concept'] == new_concept].index[0], 'TERM_ID']

            print(f'new_concept = {new_concept}')
    

## Review Unmatched Results:

In [ ]:
# read in the unmatched results
post_df_meda_no_match = df[df['SUBJECT_ID_x'].isna()]

In [ ]:
unmatched_concepts_by_color = post_df_meda_no_match.groupby(['subjects'])['media'].agg('count').reset_index()
display(unmatched_concepts_by_color.sort_values('media', ascending=False))

## Develop the flow for this to work in an interation

In [ ]:
# merge the df with the AAT Metadata 
df = pd.merge(df, concepts, how="left", on='join_concept')

In [ ]:
# get the boxid
df['boxid_temp'] = df['media'].str.extract(r"(_0\d+_)")
df = df[df['media'].notna()]
df['boxid_temp'] = df['boxid_temp'].str.replace("_","")
df['boxid_temp'] = pd.to_numeric(df['boxid_temp'], errors='coerce')
df = df.dropna(subset=['boxid_temp'])
df['boxid_temp'] = df['boxid_temp'].astype('int')
df['boxid_temp'] = df['boxid_temp'].astype('str')
df['BoxId'] = 'Box '+df['boxid_temp']
df.drop('boxid_temp', inplace=True, axis=1)

In [ ]:
# add the colors in 
color_match_df = pd.read_csv('csv_files/color_match', sep='\t')
df = pd.merge(df, color_match_df, on='BoxId')

In [ ]:
# get all the unique media IDs where media is not NA 
df_medias = df.media.dropna().unique()
df_medias

In [ ]:
# get the index of the search_media term in the array
search_media_index = df_medias.tolist().index(search_media)
search_media_index

In [ ]:
# get the list of all the subjects for the subset of the dataframe for the box
df_subjects = df[df['media'] == search_media].join_concept.to_list()
df_subjects

In [ ]:
# get the list of unmatched subjects
df_meda_no_match = df[df['media'] == search_media].loc[df['SUBJECT_ID_x'].isna()]
df_meda_no_match[narrow_list]

In [ ]:
# for any unmatched terms, check to see if combining the terms yields a match
no_match = df_meda_no_match.join_concept.to_list()
no_match

In [ ]:
# identify the index value of the ummatched item
for i, unmatched in enumerate(no_match):
    no_match_index = df_subjects.index(unmatched)
    print(f'{i+1}: unmatched concept is: "{unmatched}" and its index is "{no_match_index}" out of {len(df_subjects)-1}')

In [ ]:
no_match_index = df_subjects.index(no_match[1])
no_match_index

In [ ]:
try_match_pos = df_subjects[no_match_index]+', '+df_subjects[no_match_index+1]
try_match_pos

In [ ]:
try_match_neg = None
try_match_pos = None
concepts_found_pos = []
concepts_found_neg = []
pos = False
neg = False

if no_match_index > 0:
    try_match_neg = df_subjects[no_match_index-1]+', '+df_subjects[no_match_index]
    concepts_found_pos = concepts.loc[concepts['join_concept'] == try_match_neg]
    pos = True
if no_match_index < len(df_subjects)-1:
    try_match_pos = df_subjects[no_match_index]+', '+df_subjects[no_match_index+1]
    concepts_found_neg = concepts.loc[concepts['join_concept'] == try_match_pos]
    neg = True
print(f'try_match_neg = "{try_match_neg}"\ntry_match_pos = "{try_match_pos}"')

In [ ]:
neg

In [ ]:
concepts_found_pos[['SUBJECT_ID_x', 'TERM_ID', 'use_concept', 'join_concept']]

In [ ]:
try:
    concepts_found_neg[['SUBJECT_ID_x', 'TERM_ID', 'use_concept', 'join_concept']]
except (NameError):
    print(f'NameError: concepts_found_neg not defined')

In [ ]:
concepts_found_neg[['SUBJECT_ID_x', 'TERM_ID', 'use_concept', 'join_concept']]

In [ ]:

try:
    if len(concepts_found_pos) == 1:
        new_concept = concepts_found_pos.join_concept.to_list()[0]
except (NameError):
    print(f'positive, but NameError')
try:
    if len(concepts_found_neg) == 1:
        new_concept = concepts_found_neg.join_concept.to_list()[0]
except (NameError):
    print(f'negative, but NameError')
try:
    if (len(concepts_found_pos) > 0) & (len(concepts_found_neg) > 0):
        print('too many matches!')
except (NameError):
    pass

new_concept

In [ ]:
# determine the remove concept
remove_con = df_subjects[no_match_index]
remove_con

In [ ]:
# before the drop
df[df['media'] == search_media][narrow_list]

In [ ]:
remove_this_con_index = 0
if pos:
    remove_this_con_index = no_match_index-1
if neg:
    remove_this_con_index = no_match_index+1

remove_this_con = df_subjects[remove_this_con_index]

df = df.drop(
    df[((df['media'] == search_media) & 
                      (df['join_concept'] == remove_this_con)
                     )].index
)

In [ ]:
# after the drop

df[df['media'] == search_media][narrow_list]

In [ ]:
# replace the no_match value with the combined_concept

pop_index = df.loc[(df['media'] == search_media) & (df['join_concept'] == remove_con)].index
pop_index

In [ ]:
df.at[pop_index[0], 'join_concept'] = new_concept
df.at[pop_index[0], 'subjects'] = new_concept
df.at[pop_index[0], 'SUBJECT_ID_x'] = concepts.at[concepts.loc[concepts['join_concept'] == new_concept].index[0], 'SUBJECT_ID_x']
df.at[pop_index[0], 'TERM_ID'] = concepts.at[concepts.loc[concepts['join_concept'] == new_concept].index[0], 'TERM_ID']

In [ ]:
df.loc[df['media']== search_media][narrow_list]